# Подготовка данных

### Импорт библиотек

In [220]:
import re
import numpy as np
import pandas as pd

In [221]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Рестораны

In [222]:
df_restaurants = pd.read_json('../mining/restaurants_new.json')

In [223]:
df_restaurants.shape

(10082, 12)

In [224]:
for i in range(1, 6):
    column_name = '{}_star'.format(i)
    df_restaurants[column_name] = df_restaurants['assessments_info'].apply(lambda x: x.get(column_name)).\
        str.encode('ascii', 'ignore').str.decode('utf-8').astype(float).fillna(0).astype(int)

In [225]:
df_restaurants['review_count'] = df_restaurants['review_count'].str.encode('ascii', 'ignore').str.decode('utf-8').replace('', '0').astype(int)

In [226]:
df_restaurants['rating'] = df_restaurants['rating'].str.encode('ascii', 'ignore').str.decode('utf-8').str.replace(',', '.').replace('', '0').astype(float)

In [227]:
stop_words = [
    'Завтрак',
    'Завтрак типа "шведский стол"',
    'Обед', 
    'Ужин', 
    'Принимаются только наличные', 
    'Утвержденная парковка', 
    'Электронные платежи',
    'Бесплатный Wi-Fi', 
    'Места для сидения',
    'Обслуживание посетителей за\xa0столиками',
    'Принимаются карты Mastercard',
    'Принимаются карты Visa',
    'Для посетителей на автомобилях',
    'Имеется парковка', 
    'Парковка на улице', 
    'Телевизор',
    'Принимаются кредитные карты',
    'Для\xa0гостей с\xa0собаками',
]

In [228]:
df_restaurants['kitchen'] = df_restaurants['locality'].str.split(',').apply(lambda x: [w.strip() for w in x]).\
    apply(lambda x: [w for w in x if w not in stop_words])

In [229]:
df_restaurants['kitchen'].explode().unique()

array(['Средиземноморская', 'Русская', 'Кафе', 'Барбекю', 'Гриль',
       'Восточноевропейская', 'Европейская', 'Центральноевропейская',
       'Итальянская', 'Пицца', 'Морепродукты', 'Бельгийская', 'Бар',
       'Французская', 'Испанская', 'Паб', 'Гастропаб', 'Современная',
       'Японская', 'Азиатская', 'Супы', 'Стейк-хаус', 'Американская',
       'Напитки', 'Подходит для вегетарианцев', 'Для веганов',
       'Безглютеновые блюда', 'Фастфуд', 'Фьюжн', 'Кавказская',
       'Грузинская', 'Винный бар', 'Украинская', 'Аргентинская',
       'Здоровая', 'Каталанская', 'Международная', 'Тосканская',
       'Центрально-итальянская', 'Неаполитанская', 'Кухня Кампании',
       'Ирландская', 'Британская', 'Шотландская', 'Суши',
       'Южно-итальянская', 'Шведская', 'Скандинавская', 'Датская',
       'Норвежская', 'Кошерная', 'Немецкая', 'Закусочная',
       'Открыто допоздна', 'Бранч', 'Карибская', 'Гавайская', 'Ямайская',
       'Корейская', 'Кухня Хоккайдо', 'Филиппинская', 'Ближневосточная

In [230]:
df_restaurants['price'] = df_restaurants['price'].str.encode('ascii', 'ignore').str.decode('utf-8').str.replace(',', '').\
    str.replace(' ', '').str.split('-').apply(lambda x: [w.strip() for w in x])

In [231]:
df_restaurants['price_min'] = df_restaurants['price'].apply(lambda x: int(x[0]) if x[0] != '' else 0)
df_restaurants['price_max'] = df_restaurants['price'].apply(lambda x: int(x[1]) if x[0] != '' else 0)

In [232]:
df_restaurants['geodata'][0]

' Местоположение и контактные данныеул. 1905 года, 10/1, Москва 123022 РоссияПресненский район1,3 км  от: Московский зоопаркСайтЭлектронная почта+7 926 222-71-22Исправить данные об объекте'

In [233]:
restaurants = df_restaurants['title'].unique().tolist()

In [240]:
# restaurants

In [235]:
df_restaurants['rest_id'] = df_restaurants['url'][0].split('-')[1] + '-' + df_restaurants['url'][0].split('-')[2]

In [236]:
columns = [
    'rest_id',
    'url',
    'title',
    'rating',
    'review_count',
    'street_adress',
    'phone_number',
    '1_star',
    '2_star',
    '3_star',
    '4_star',
    '5_star',
    'kitchen',
    'price_min',
    'price_max',
]

In [237]:
df_restaurants[columns].head()

,rest_id,url,title,rating,review_count,street_adress,phone_number,1_star,2_star,3_star,4_star,5_star,kitchen,price_min,price_max
0,g298484-d8429267,https://www.tripadvisor.ru//Restaurant_Review-...,Сабор де ла Вида Ресторан,5.0,323,"ул. 1905 года, 10/1, Москва 123022 Россия",+7 926 222-71-22,2,2,7,16,275,[Средиземноморская],280,2500
1,g298484-d8429267,https://www.tripadvisor.ru//Restaurant_Review-...,Любовь Пирогова,5.0,138,"ул. Мытная, 74 Даниловский рынок, Москва 11519...",+7 495 120-00-97,2,3,3,4,122,"[Русская, Кафе]",150,500
2,g298484-d8429267,https://www.tripadvisor.ru//Restaurant_Review-...,Джумбус,5.0,320,"ул. Добровольческая 12, Москва 109004 Россия",+7 903 590-34-70,4,2,9,11,279,"[Средиземноморская, Барбекю, Гриль, Восточноев...",100,1000
3,g298484-d8429267,https://www.tripadvisor.ru//Restaurant_Review-...,Остерия Марио,4.5,319,"ул. Балтийская, 9, Москва 125315 Россия",+7 968 681-18-66,5,7,19,37,238,"[Итальянская, Пицца, Морепродукты, Средиземном...",800,1200
4,g298484-d8429267,https://www.tripadvisor.ru//Restaurant_Review-...,Leffe Cafe,5.0,138,"Расторгуевский переулок, дом 3А, Москва 123557...",+7 915 323-73-30,0,2,6,9,112,"[Европейская, Бельгийская, Бар, Кафе]",500,1400


In [239]:
df_restaurants[columns].to_feather('../data/restaurants.feather')

# Обзоры

In [241]:
df_reviews = pd.read_json('../mining/reviews.json')

In [242]:
df_reviews.shape

(332139, 8)

In [245]:
df_reviews['rest_id'] = df_reviews['url'][0].split('-')[1] + '-' + df_reviews['url'][0].split('-')[2]

In [247]:
df_reviews['review_id'] = df_reviews['url'][0].split('-')[1] + '-' + df_reviews['url'][0].split('-')[2] + '-' + df_reviews['url'][0].split('-')[3]

In [255]:
rev_columns = [
    'rest_id',
    'review_id',
    'restaurant_name',
    'url',
    'username',
    'date',
    'review_title',
    'review_body',
    'date_of_visit',
    'stars_amount',
]

In [256]:
df_reviews[rev_columns].head()

,rest_id,review_id,restaurant_name,url,username,date,review_title,review_body,date_of_visit,stars_amount
0,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,olgusik1995,Отзыв написан неделю назад,Вкусный визит,Вкусно) очень вкусно попробовали несколько раз...,Дата посещения: декабрь 2021 г.,5
1,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,alexsW1908FJ,Отзыв написан неделю назад,Необычные сочетания продуктов в начинке,Необычные сочетания продуктов в начинке. Безус...,Дата посещения: декабрь 2021 г.,5
2,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,_K6782NJ,Отзыв написан неделю назад,Лучшие пироги тут,Впервые были тут и попробовали пироги. Нежнейш...,Дата посещения: декабрь 2021 г.,5
3,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,Alexey_Kalinin,Отзыв написан 2 недели назад,Лучшие пироги в Москве,Домой с доставкой заказали. Как всегда идеальн...,Дата посещения: декабрь 2021 г.,5
4,g298484-d10841351,g298484-d10841351-r823334817,Любовь Пирогова,https://www.tripadvisor.ru//ShowUserReviews-g2...,803evgeniias,Отзыв написан 2 недели назад,Доставка наивкуснейших пирогов,Без притворства это наивкуснейшие пироги! Зака...,Дата посещения: декабрь 2021 г.,5


In [258]:
df_reviews[rev_columns].to_feather('../data/reviews.feather')